In [1]:
from marubatsu import Marubatsu, ListBoard

def __init__(self, board_size=3, count_linemark=False):
    self.BOARD_SIZE = board_size
    self.count_linemark = count_linemark
    self.board = [[Marubatsu.EMPTY] * self.BOARD_SIZE for y in range(self.BOARD_SIZE)]
    if self.count_linemark:
        self.rowcount = {
            Marubatsu.CIRCLE: [0] * self.BOARD_SIZE,
            Marubatsu.CROSS: [0] * self.BOARD_SIZE,
        }
        self.colcount = {
            Marubatsu.CIRCLE: [0] * self.BOARD_SIZE,
            Marubatsu.CROSS: [0] * self.BOARD_SIZE,
        }
        self.diacount = {
            Marubatsu.CIRCLE: [0] * 2,
            Marubatsu.CROSS: [0] * 2,
        }
        
ListBoard.__init__  = __init__

In [2]:
def setmark(self, x, y, mark):
    if self.count_linemark:
        if mark != Marubatsu.EMPTY:
            diff = 1
            countmark = mark
        else:
            diff = -1
            countmark = self.board[x][y]
        self.colcount[countmark][x] += diff
        self.rowcount[countmark][y] += diff
        if x == y:
            self.diacount[countmark][0] += diff
        if x + y == self.BOARD_SIZE - 1:
            self.diacount[countmark][1] += diff
    self.board[x][y] = mark

ListBoard.setmark = setmark

In [3]:
def judge(self, last_turn, last_move, move_count):
    if move_count < self.BOARD_SIZE * 2 - 1:
        return Marubatsu.PLAYING
    # 直前に着手を行ったプレイヤーの勝利の判定
    if self.is_winner(last_turn, last_move):
        return last_turn
    # 引き分けの判定
    elif move_count == self.BOARD_SIZE ** 2:
        return Marubatsu.DRAW
    # 上記のどれでもなければ決着がついていない
    else:
        return Marubatsu.PLAYING   

ListBoard.judge = judge

In [4]:
def is_winner(self, player, last_move):
    x, y = last_move
    if self.count_linemark:
        if self.rowcount[player][y] == self.BOARD_SIZE or \
        self.colcount[player][x] == self.BOARD_SIZE:
            return True
        # 左上から右下方向の判定
        if x == y and self.diacount[player][0] == self.BOARD_SIZE:
            return True
        # 右上から左下方向の判定
        if x + y == self.BOARD_SIZE - 1 and \
            self.diacount[player][1] == self.BOARD_SIZE:
            return True
    else:
        if self.is_same(player, coord=[0, y], dx=1, dy=0) or \
        self.is_same(player, coord=[x, 0], dx=0, dy=1):
            return True
        # 左上から右下方向の判定
        if x == y and self.is_same(player, coord=[0, 0], dx=1, dy=1):
            return True
        # 右上から左下方向の判定
        if x + y == self.BOARD_SIZE - 1 and \
            self.is_same(player, coord=[self.BOARD_SIZE - 1, 0], dx=-1, dy=1):
            return True
    
    # どの一直線上にも配置されていない場合は、player は勝利していないので False を返す
    return False

ListBoard.is_winner = is_winner

In [5]:
def is_same(self, mark, coord, dx, dy):   
    x, y = coord
    text_list = [self.getmark(x + i * dx, y + i * dy)  
                    for i in range(self.BOARD_SIZE)]
    line_text = "".join(text_list)
    return line_text == mark * self.BOARD_SIZE

ListBoard.is_same = is_same

In [6]:
lb = ListBoard(count_linemark=False)
movelist = [(0, 0), (1, 1), (1, 0), (2, 2), (2, 0)]
turn = Marubatsu.CIRCLE
move_count = 0
for x, y in movelist:
    print(f"({x}, {y}) に {turn} を着手")
    lb.setmark(x, y, turn)
    move_count += 1
    last_move = (x, y)
    last_turn = turn
    turn = Marubatsu.CROSS if turn == Marubatsu.CIRCLE else Marubatsu.CIRCLE
    print(lb.board)
    print(lb.judge(last_turn, last_move, move_count))
    print()

(0, 0) に o を着手
[['o', '.', '.'], ['.', '.', '.'], ['.', '.', '.']]
playing

(1, 1) に x を着手
[['o', '.', '.'], ['.', 'x', '.'], ['.', '.', '.']]
playing

(1, 0) に o を着手
[['o', '.', '.'], ['o', 'x', '.'], ['.', '.', '.']]
playing

(2, 2) に x を着手
[['o', '.', '.'], ['o', 'x', '.'], ['.', '.', 'x']]
playing

(2, 0) に o を着手
[['o', '.', '.'], ['o', 'x', '.'], ['o', '.', 'x']]
o



In [7]:
lb = ListBoard(count_linemark=True)
movelist = [(0, 0), (1, 1), (1, 0), (2, 2), (2, 0)]
turn = Marubatsu.CIRCLE
move_count = 0
for x, y in movelist:
    print(f"({x}, {y}) に {turn} を着手")
    lb.setmark(x, y, turn)
    move_count += 1
    last_move = (x, y)
    last_turn = turn
    turn = Marubatsu.CROSS if turn == Marubatsu.CIRCLE else Marubatsu.CIRCLE
    print(lb.board)
    print("行", lb.rowcount)
    print("列", lb.colcount)   
    print("斜め", lb.diacount)
    print(lb.judge(last_turn, last_move, move_count))
    print()

(0, 0) に o を着手
[['o', '.', '.'], ['.', '.', '.'], ['.', '.', '.']]
行 {'o': [1, 0, 0], 'x': [0, 0, 0]}
列 {'o': [1, 0, 0], 'x': [0, 0, 0]}
斜め {'o': [1, 0], 'x': [0, 0]}
playing

(1, 1) に x を着手
[['o', '.', '.'], ['.', 'x', '.'], ['.', '.', '.']]
行 {'o': [1, 0, 0], 'x': [0, 1, 0]}
列 {'o': [1, 0, 0], 'x': [0, 1, 0]}
斜め {'o': [1, 0], 'x': [1, 1]}
playing

(1, 0) に o を着手
[['o', '.', '.'], ['o', 'x', '.'], ['.', '.', '.']]
行 {'o': [2, 0, 0], 'x': [0, 1, 0]}
列 {'o': [1, 1, 0], 'x': [0, 1, 0]}
斜め {'o': [1, 0], 'x': [1, 1]}
playing

(2, 2) に x を着手
[['o', '.', '.'], ['o', 'x', '.'], ['.', '.', 'x']]
行 {'o': [2, 0, 0], 'x': [0, 1, 1]}
列 {'o': [1, 1, 0], 'x': [0, 1, 1]}
斜め {'o': [1, 0], 'x': [2, 1]}
playing

(2, 0) に o を着手
[['o', '.', '.'], ['o', 'x', '.'], ['o', '.', 'x']]
行 {'o': [3, 0, 0], 'x': [0, 1, 1]}
列 {'o': [1, 1, 1], 'x': [0, 1, 1]}
斜め {'o': [1, 1], 'x': [2, 1]}
o



In [8]:
for x, y in movelist:
    print(f"({x}, {y}) から {lb.getmark(x, y)} を削除")    
    lb.setmark(x, y, Marubatsu.EMPTY)
    print(lb.board)
    print("行", lb.rowcount)
    print("列", lb.colcount)   
    print("斜め", lb.diacount)
    print()

(0, 0) から o を削除
[['.', '.', '.'], ['o', 'x', '.'], ['o', '.', 'x']]
行 {'o': [2, 0, 0], 'x': [0, 1, 1]}
列 {'o': [0, 1, 1], 'x': [0, 1, 1]}
斜め {'o': [0, 1], 'x': [2, 1]}

(1, 1) から x を削除
[['.', '.', '.'], ['o', '.', '.'], ['o', '.', 'x']]
行 {'o': [2, 0, 0], 'x': [0, 0, 1]}
列 {'o': [0, 1, 1], 'x': [0, 0, 1]}
斜め {'o': [0, 1], 'x': [1, 0]}

(1, 0) から o を削除
[['.', '.', '.'], ['.', '.', '.'], ['o', '.', 'x']]
行 {'o': [1, 0, 0], 'x': [0, 0, 1]}
列 {'o': [0, 0, 1], 'x': [0, 0, 1]}
斜め {'o': [0, 1], 'x': [1, 0]}

(2, 2) から x を削除
[['.', '.', '.'], ['.', '.', '.'], ['o', '.', '.']]
行 {'o': [1, 0, 0], 'x': [0, 0, 0]}
列 {'o': [0, 0, 1], 'x': [0, 0, 0]}
斜め {'o': [0, 1], 'x': [0, 0]}

(2, 0) から o を削除
[['.', '.', '.'], ['.', '.', '.'], ['.', '.', '.']]
行 {'o': [0, 0, 0], 'x': [0, 0, 0]}
列 {'o': [0, 0, 0], 'x': [0, 0, 0]}
斜め {'o': [0, 0], 'x': [0, 0]}



In [9]:
def __init__(self, boardclass=ListBoard, board_size=3, check_coord=True, *args, **kwargs):
    # ゲーム盤のデータ構造を定義するクラス
    self.boardclass = boardclass
    # ゲーム盤の縦横のサイズ
    self.BOARD_SIZE = board_size
    # boardclass のパラメータ
    self.args = args
    self.kwargs = kwargs
    # move と unmove メソッドで座標などのチェックを行うかどうか
    self.check_coord = check_coord
    # 〇×ゲーム盤を再起動するメソッドを呼び出す
    self.restart()
    
Marubatsu.__init__ = __init__

In [10]:
def initialize_board(self):
    self.board = self.boardclass(self.BOARD_SIZE, *self.args, **self.kwargs)
    
Marubatsu.initialize_board = initialize_board

In [11]:
def restart(self):
    self.initialize_board()
    self.turn = Marubatsu.CIRCLE     
    self.move_count = 0
    self.status = Marubatsu.PLAYING
    self.last_move = -1, -1          
    self.last_turn = None
    self.records = [self.last_move]

Marubatsu.restart = restart

In [12]:
def move(self, x, y):
    if self.place_mark(x, y, self.turn):
        self.last_turn = self.turn
        self.turn = Marubatsu.CROSS if self.turn == Marubatsu.CIRCLE else Marubatsu.CIRCLE  
        self.move_count += 1
        self.last_move = x, y
        self.status = self.board.judge(self.last_turn, self.last_move, self.move_count)
        if len(self.records) <= self.move_count:            
            self.records.append(self.last_move)
        else:
            self.records[self.move_count] = self.last_move
            self.records = self.records[0:self.move_count + 1]
            
Marubatsu.move = move

In [13]:
def unmove(self):
    if self.move_count > 0:
        x, y = self.last_move
        if self.move_count == 0:
            self.last_move = (-1, -1)
        self.move_count -= 1
        self.turn, self.last_turn = self.last_turn, self.turn
        self.status = Marubatsu.PLAYING
        x, y = self.records.pop()
        self.remove_mark(x, y)       
        self.last_move = self.records[-1]
        
Marubatsu.unmove = unmove  

In [14]:
def judge(self):
    return self.board.judge(self.last_turn, self.last_move, self.move_count)

Marubatsu.judge = judge

In [15]:
from ai import ai_match, ai2s

ai_match(ai=[ai2s, ai2s], match_num=5000, 
         mbparams={"count_linemark": False, "check_coord": True})
ai_match(ai=[ai2s, ai2s], match_num=5000, 
         mbparams={"count_linemark": True, "check_coord": True})
ai_match(ai=[ai2s, ai2s], match_num=5000, 
         mbparams={"count_linemark": False, "check_coord": False})
ai_match(ai=[ai2s, ai2s], match_num=5000, 
         mbparams={"count_linemark": True, "check_coord": False})


ai2s VS ai2s


100%|██████████| 5000/5000 [00:01<00:00, 2540.72it/s]


count     win    lose    draw
o        2927    1440     633
x        1435    2915     650
total    4362    4355    1283

ratio     win    lose    draw
o       58.5%   28.8%   12.7%
x       28.7%   58.3%   13.0%
total   43.6%   43.5%   12.8%

ai2s VS ai2s


100%|██████████| 5000/5000 [00:01<00:00, 2590.08it/s]


count     win    lose    draw
o        2962    1416     622
x        1473    2891     636
total    4435    4307    1258

ratio     win    lose    draw
o       59.2%   28.3%   12.4%
x       29.5%   57.8%   12.7%
total   44.4%   43.1%   12.6%

ai2s VS ai2s


100%|██████████| 5000/5000 [00:01<00:00, 2717.71it/s]


count     win    lose    draw
o        2907    1427     666
x        1412    2962     626
total    4319    4389    1292

ratio     win    lose    draw
o       58.1%   28.5%   13.3%
x       28.2%   59.2%   12.5%
total   43.2%   43.9%   12.9%

ai2s VS ai2s


100%|██████████| 5000/5000 [00:01<00:00, 2780.95it/s]

count     win    lose    draw
o        2934    1452     614
x        1394    2990     616
total    4328    4442    1230

ratio     win    lose    draw
o       58.7%   29.0%   12.3%
x       27.9%   59.8%   12.3%
total   43.3%   44.4%   12.3%



[('count',
  [{'win': 2934, 'lose': 1452, 'draw': 614},
   {'win': 1394, 'lose': 2990, 'draw': 616},
   {'win': 4328, 'lose': 4442, 'draw': 1230}],
  '7d'),
 ('ratio',
  [{'win': 0.5868, 'lose': 0.2904, 'draw': 0.1228},
   {'win': 0.2788, 'lose': 0.598, 'draw': 0.1232},
   {'win': 0.4328, 'lose': 0.4442, 'draw': 0.123}],
  '7.1%')]

In [16]:
from mbtest import excel_to_xy

def test_judge(testcases=None, debug=False, mbparams={}):
    if testcases is None:
        testcases = {
            # 決着がついていない場合のテストケース
            Marubatsu.PLAYING: [
                # ゲーム盤に一つもマークが配置されていない場合のテストケース
                "",
                # 一つだけマークが配置されていない場合のテストケース
                "C3,A2,B1,B2,C2,C1,A3,B3",
                "A1,A2,C3,B2,C2,C1,A3,B3",
                "A1,A2,B1,B2,C2,C3,A3,B3",
                "A1,C3,B1,B2,C2,C1,A3,B3",
                "A1,A2,B1,C3,C2,C1,A3,B3",
                "A1,A2,B1,B2,C3,C1,A3,B3",
                "A1,A2,B1,B2,C2,C1,C3,B3",
                "A1,A2,B1,B2,A3,C1,C2,C3",
                "A1,A2,B1,B2,C2,C1,A3,B3",
            ],   
            # 〇の勝利のテストケース
            Marubatsu.CIRCLE: [
                "A1,A2,B1,B2,C1",
                "A2,A1,B2,B1,C2",
                "A3,A1,B3,B1,C3",
                "A1,B1,A2,B2,A3",
                "B1,A1,B2,A2,B3",
                "C1,A1,C2,A2,C3",
                "A1,A2,B2,A3,C3",
                "A3,A1,B2,A2,C1",
                # 簡易的な組み合わせ網羅の 6 のテストケース
                "A1,B1,A2,B2,B3,C1,C3,C2,A3", 
            ],
            # × の勝利のテストケース
            Marubatsu.CROSS: [
                "A2,A1,B2,B1,A3,C1",
                "A1,A2,B1,B2,A3,C2",
                "A1,A3,B1,B3,A2,C3",
                "B1,A1,B2,A2,C1,A3",
                "A1,B1,A2,B2,C1,B3",
                "A1,C1,A2,C2,B1,C3",
                "A2,A1,A3,B2,B1,C3",
                "A1,C1,B1,B2,A2,A3",
            ],
            # 引き分けの場合のテストケース
            Marubatsu.DRAW: [
                "A1,A2,B1,B2,C2,C1,A3,B3,C3",
            ], 
        }
            
    print("Start")
    for winner, testdata_list in testcases.items():
        print("test winner =", winner) 
        for testdata in testdata_list:
            mb = Marubatsu(**mbparams)
            for coord in [] if testdata == "" else testdata.split(","):
                x, y = excel_to_xy(coord)            
                mb.move(x, y)
            if debug:
                print(mb)

            if mb.judge() == winner:
                if debug:
                    print("ok")
                else:
                    print("o", end="")
            else:
                print()
                print("====================")
                print("test_judge error!")
                print(mb)
                print("mb.judge():", mb.judge())
                print("winner:    ", winner)
                print("====================")
        print()
    print("Finished")

In [17]:
test_judge(mbparams={"count_linemark": False})
test_judge(mbparams={"count_linemark": True})

Start
test winner = playing
oooooooooo
test winner = o
ooooooooo
test winner = x
oooooooo
test winner = draw
o
Finished
Start
test winner = playing
oooooooooo
test winner = o
ooooooooo
test winner = x
oooooooo
test winner = draw
o
Finished


In [18]:
from marubatsu import Markpat
from collections import defaultdict

def count_markpats(self, turn, last_turn):
    markpats = defaultdict(int)
    
    if self.count_linemark:
        for countdict in [self.rowcount, self.colcount, self.diacount]:
            for circlecount, crosscount in zip(countdict[Marubatsu.CIRCLE], countdict[Marubatsu.CROSS]):
                emptycount = self.BOARD_SIZE - circlecount - crosscount
                if last_turn == Marubatsu.CIRCLE:
                    markpats[(circlecount, crosscount, emptycount)] += 1
                else:
                    markpats[(crosscount, circlecount, emptycount)] += 1
    else:
        # 横方向と縦方向の判定
        for i in range(self.BOARD_SIZE):
            count = self.count_marks(turn, last_turn, coord=[0, i], dx=1, dy=0, datatype="tuple")
            markpats[count] += 1
            count = self.count_marks(turn, last_turn, coord=[i, 0], dx=0, dy=1, datatype="tuple")
            markpats[count] += 1
        # 左上から右下方向の判定
        count = self.count_marks(turn, last_turn, coord=[0, 0], dx=1, dy=1, datatype="tuple")
        markpats[count] += 1
        # 右上から左下方向の判定
        count = self.count_marks(turn, last_turn, coord=[2, 0], dx=-1, dy=1, datatype="tuple")
        markpats[count] += 1

    return markpats   

ListBoard.count_markpats = count_markpats

In [19]:
def count_marks(self, turn, last_turn, coord, dx, dy, datatype="dict"):   
    x, y = coord   
    count = defaultdict(int)
    for _ in range(self.BOARD_SIZE):
        count[self.getmark(x, y)] += 1
        x += dx
        y += dy

    if datatype == "dict":
        return count
    else:
        return Markpat(count[last_turn], count[turn], count[Marubatsu.EMPTY])  
    
ListBoard.count_marks = count_marks

In [20]:
def count_markpats(self):
    return self.board.count_markpats(self.turn, self.last_turn)

Marubatsu.count_markpats = count_markpats

In [21]:
from ai import ai_by_score

@ai_by_score
def ai8s(mb, debug=False):
    # 真ん中のマスに着手している場合は、評価値として 3 を返す
    if mb.last_move == (1, 1):
        return 3

    # 自分が勝利している場合は、評価値として 2 を返す
    if mb.status == mb.last_turn:
        return 2

    markpats = mb.count_markpats()
    # 相手が勝利できる場合は評価値として -1 を返す
    if markpats[Markpat(last_turn=0, turn=2, empty=1)] > 0:
        return -1
    # 次の自分の手番で自分が勝利できる場合は評価値として 1 を返す
    elif markpats[Markpat(last_turn=2, turn=0, empty=1)] > 0:
        return 1
    # それ以外の場合は評価値として 0 を返す
    else:
        return 0

In [22]:
from ai import ai14s

ai_match(ai=[ai8s, ai2s], mbparams={"count_linemark": False})
ai_match(ai=[ai8s, ai2s], mbparams={"count_linemark": True})
ai_match(ai=[ai14s, ai2s], mbparams={"count_linemark": False})
ai_match(ai=[ai14s, ai2s], mbparams={"count_linemark": True})

ai8s VS ai2s


100%|██████████| 10000/10000 [00:10<00:00, 938.44it/s]


count     win    lose    draw
o        9852      14     134
x        8953     226     821
total   18805     240     955

ratio     win    lose    draw
o       98.5%    0.1%    1.3%
x       89.5%    2.3%    8.2%
total   94.0%    1.2%    4.8%

ai8s VS ai2s


100%|██████████| 10000/10000 [00:05<00:00, 1784.56it/s]


count     win    lose    draw
o        9858      19     123
x        8856     225     919
total   18714     244    1042

ratio     win    lose    draw
o       98.6%    0.2%    1.2%
x       88.6%    2.2%    9.2%
total   93.6%    1.2%    5.2%

ai14s VS ai2s


100%|██████████| 10000/10000 [00:11<00:00, 858.05it/s]


count     win    lose    draw
o        9896       0     104
x        8849       0    1151
total   18745       0    1255

ratio     win    lose    draw
o       99.0%    0.0%    1.0%
x       88.5%    0.0%   11.5%
total   93.7%    0.0%    6.3%

ai14s VS ai2s


100%|██████████| 10000/10000 [00:06<00:00, 1449.95it/s]

count     win    lose    draw
o        9915       0      85
x        8774       0    1226
total   18689       0    1311

ratio     win    lose    draw
o       99.2%    0.0%    0.9%
x       87.7%    0.0%   12.3%
total   93.4%    0.0%    6.6%



[('count',
  [{'win': 9915, 'lose': 0, 'draw': 85},
   {'win': 8774, 'lose': 0, 'draw': 1226},
   {'win': 18689, 'lose': 0, 'draw': 1311}],
  '7d'),
 ('ratio',
  [{'win': 0.9915, 'lose': 0.0, 'draw': 0.0085},
   {'win': 0.8774, 'lose': 0.0, 'draw': 0.1226},
   {'win': 0.93445, 'lose': 0.0, 'draw': 0.06555}],
  '7.1%')]

In [ ]:
from marubatsu import Board
from abc import abstractmethod

@abstractmethod
def judge(self, last_turn, last_move, move_count):
    pass

Board.judge = judge

@abstractmethod
def count_markpats(self, turn, last_turn):
    pass

Board.count_markpat = count_markpats

In [24]:
class List1dBoard(ListBoard):
    def __init__(self, board_size=3, count_linemark=False):
        self.BOARD_SIZE = board_size
        self.count_linemark = count_linemark
        self.board = [Marubatsu.EMPTY] * (self.BOARD_SIZE ** 2)
        if self.count_linemark:
            self.rowcount = {
                Marubatsu.CIRCLE: [0] * self.BOARD_SIZE,
                Marubatsu.CROSS: [0] * self.BOARD_SIZE,
            }
            self.colcount = {
                Marubatsu.CIRCLE: [0] * self.BOARD_SIZE,
                Marubatsu.CROSS: [0] * self.BOARD_SIZE,
            }
            self.diacount = {
                Marubatsu.CIRCLE: [0] * 2,
                Marubatsu.CROSS: [0] * 2,
            }
            
    def setmark(self, x, y, mark):
        if self.count_linemark:
            if mark != Marubatsu.EMPTY:
                diff = 1
                changedmark = mark
            else:
                diff = -1
                changedmark = self.board[y + x * self.BOARD_SIZE]
            self.colcount[changedmark][x] += diff
            self.rowcount[changedmark][y] += diff
            if x == y:
                self.diacount[changedmark][0] += diff
            if x + y == self.BOARD_SIZE - 1:
                self.diacount[changedmark][1] += diff
        self.board[y + x * self.BOARD_SIZE] = mark

    def getmark(self, x, y):
        return self.board[y + x * self.BOARD_SIZE]

    def board_to_str(self):
        return "".join(self.board)

In [25]:
ai_match(ai=[ai8s, ai2s], 
         mbparams={"boardclass":List1dBoard, "count_linemark": False})
ai_match(ai=[ai8s, ai2s], 
         mbparams={"boardclass":List1dBoard, "count_linemark": True})
ai_match(ai=[ai14s, ai2s], 
         mbparams={"boardclass":List1dBoard, "count_linemark": False})
ai_match(ai=[ai14s, ai2s], 
         mbparams={"boardclass":List1dBoard, "count_linemark": True})


ai8s VS ai2s


100%|██████████| 10000/10000 [00:10<00:00, 972.90it/s]


count     win    lose    draw
o        9851       8     141
x        8930     249     821
total   18781     257     962

ratio     win    lose    draw
o       98.5%    0.1%    1.4%
x       89.3%    2.5%    8.2%
total   93.9%    1.3%    4.8%

ai8s VS ai2s


100%|██████████| 10000/10000 [00:05<00:00, 1711.13it/s]


count     win    lose    draw
o        9849      10     141
x        8929     249     822
total   18778     259     963

ratio     win    lose    draw
o       98.5%    0.1%    1.4%
x       89.3%    2.5%    8.2%
total   93.9%    1.3%    4.8%

ai14s VS ai2s


100%|██████████| 10000/10000 [00:12<00:00, 810.29it/s]


count     win    lose    draw
o        9910       0      90
x        8808       0    1192
total   18718       0    1282

ratio     win    lose    draw
o       99.1%    0.0%    0.9%
x       88.1%    0.0%   11.9%
total   93.6%    0.0%    6.4%

ai14s VS ai2s


100%|██████████| 10000/10000 [00:07<00:00, 1380.18it/s]

count     win    lose    draw
o        9902       0      98
x        8831       0    1169
total   18733       0    1267

ratio     win    lose    draw
o       99.0%    0.0%    1.0%
x       88.3%    0.0%   11.7%
total   93.7%    0.0%    6.3%



[('count',
  [{'win': 9902, 'lose': 0, 'draw': 98},
   {'win': 8831, 'lose': 0, 'draw': 1169},
   {'win': 18733, 'lose': 0, 'draw': 1267}],
  '7d'),
 ('ratio',
  [{'win': 0.9902, 'lose': 0.0, 'draw': 0.0098},
   {'win': 0.8831, 'lose': 0.0, 'draw': 0.1169},
   {'win': 0.93665, 'lose': 0.0, 'draw': 0.06335}],
  '7.1%')]